# PESTPP-GLM

In this notebook, we will run PESTPP-GLM in standard parameter estimation mode and regularization mode.  In both cases, we will use the baked-in bayes-linear posterior monte carlo analysis to get posterior forecast PDFs.  We will use the prior monte carlo outputs as the prior forecast PDF.

In [ ]:
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.size']=12
import flopy
import pyemu


In [ ]:
m_d = "master_glm"

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_pp.pst"))
pst.write_par_summary_table(filename="none")

In [ ]:
cov = pyemu.Cov.from_binary(os.path.join(m_d,"prior_cov.jcb")).to_dataframe()
cov = cov.loc[pst.adj_par_names,pst.adj_par_names]
cov = pyemu.Cov.from_dataframe(cov)

In [ ]:
sc = pyemu.Schur(jco=os.path.join(m_d,"freyberg_pp.jcb"),parcov=cov)

In [ ]:
df = sc.get_forecast_summary()
df

In [ ]:
df = df.loc[:,"percent_reduction"].dropna()
df.plot(kind="bar")

In [ ]:
df = sc.get_par_group_contribution()#.sort_values(by="reduction").iloc[:10].plot(kind="hist")


In [ ]:
base = df.loc["base",:]
df = 100.0 * (base - df) / base

In [ ]:
for forecast in df.columns:
    fore_df = df.loc[:,forecast].copy()
    fore_df.sort_values(inplace=True, ascending=False)
    ax = fore_df.iloc[:10].plot(kind="bar")
    ax.set_title(forecast)
    ax.set_ylabel("percent variance reduction")
    plt.show()

In [ ]:
df = sc.get_removed_obs_importance()
base = df.loc["base",:]
df = 100 * (df - base) / base
df

In [ ]:
for forecast in df.columns:
    fore_df = df.loc[:,forecast].copy()
    fore_df.sort_values(inplace=True, ascending=False)
    ax = fore_df.iloc[:10].plot(kind="bar")
    ax.set_title(forecast)
    ax.set_ylabel("percent variance increase")
    plt.show()

In [ ]:
df = sc.get_added_obs_importance()